### Onboarding Waitlist 

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # Suppress SettingWithCopyWarning

In [2]:
onboarding_data = pd.read_csv('onboarding_waitlist.csv', low_memory=False)
registration_data = pd.read_excel('network_register.xlsx', sheet_name='Sheet1')
network_waitlist_data = pd.read_excel('network_waitlist.xlsx', sheet_name='Sheet1')

print(len(onboarding_data))
print(len(registration_data))
print(len(network_waitlist_data))

284
501
268


### Prepare Data and Merge 

In [3]:
network_waitlist_data = network_waitlist_data[~network_waitlist_data["Waitlist No."].isnull()]
print(len(network_waitlist_data))

264


In [4]:
# Strip of leading and tailing white space
onboarding_data["safe_address"] = onboarding_data["safe_address"].str.strip()
registration_data["What is your HOPR safe address?"] = registration_data["What is your HOPR safe address?"].str.strip()
network_waitlist_data["Safe address"] = network_waitlist_data["Safe address"].str.strip()

# Make everything lower case letters
onboarding_data["safe_address"] = [x.lower() for x in onboarding_data["safe_address"]]
registration_data["What is your HOPR safe address?"] = [x.lower() for x in registration_data["What is your HOPR safe address?"]]
network_waitlist_data["Safe address"] = [x.lower() for x in network_waitlist_data["Safe address"]]

# Check for duplicates
dataDup_onboarding = onboarding_data.duplicated(subset=['safe_address'], keep='last')
dataDup_registration = registration_data.duplicated(subset=['What is your HOPR safe address?'], keep='last')

# dataDup.value_counts()
onboarding_data['Duplicate'] = dataDup_onboarding
registration_data['Duplicate'] = dataDup_registration

print(len(registration_data))

501


In [8]:
registration_data[registration_data["What is your HOPR safe address?"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,Time,Participant,What is your HOPR safe address?,What is your Node address,Do you already have the Network Registry NFT?,How would you like to be informed once you're able to join the network?,What is your Telegram handle?,What is your e-mail?,Duplicate
459,2023-10-02T19:23:47.388Z,landeros,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,Yes,I'll monitor the waitlist and my address manually,NaN,NaN,False


In [5]:
# Only keep unique values
onboarding_data_01 = onboarding_data.loc[onboarding_data['Duplicate'] == False]
registration_data_01 = registration_data.loc[registration_data['Duplicate'] == False]

print(onboarding_data_01['Duplicate'].value_counts())
print(registration_data_01['Duplicate'].value_counts())

Duplicate
False    284
Name: count, dtype: int64
Duplicate
False    474
Name: count, dtype: int64


In [12]:
# registration_data_01[registration_data_01["What is your HOPR safe address?"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]
onboarding_data_01[onboarding_data_01["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,deployment_date,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,nft_id,Duplicate
139,2023-10-02 18:49:25.000 UTC,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,18592.0,False


In [6]:
## Delete people who are already in the waitlist from the registration data

x = list(network_waitlist_data[network_waitlist_data['Eligibility'] == "yes"]["Safe address"])
print(len(x))

registration_data_01['waitlist_ind'] = np.where(registration_data_01["What is your HOPR safe address?"].isin(x), "YES", "No")
registration_data_02 = registration_data_01[registration_data_01['waitlist_ind'] == "No"]
print(len(registration_data_02))

150
324


In [14]:
registration_data_02[registration_data_02["What is your HOPR safe address?"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,Time,Participant,What is your HOPR safe address?,What is your Node address,Do you already have the Network Registry NFT?,How would you like to be informed once you're able to join the network?,What is your Telegram handle?,What is your e-mail?,Duplicate,waitlist_ind
459,2023-10-02T19:23:47.388Z,landeros,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,Yes,I'll monitor the waitlist and my address manually,NaN,NaN,False,No


In [7]:
waitlist = registration_data_02.merge(onboarding_data_01, how='left', left_on='What is your HOPR safe address?'
                                                    , right_on='safe_address')

waitlist = waitlist[['Time', 'deployment_date', 'What is your Node address', 'safe_address', 'deployment_tx_hash', 'wxHOPR_balance', 'nr_nft']]

# rename comumns
waitlist = waitlist.rename(columns={"Time": "registration_time", "What is your Node address": "node_address"})
print(len(waitlist))

# exlude non elidgible nodes
waitlist_01 = waitlist[~waitlist['safe_address'].isnull()]
print(len(waitlist_01))
display(waitlist_01.head())

324
133


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft
38,2023-09-09T14:31:55.044Z,2023-09-09 14:22:30.000 UTC,0x775175769897c0a46781b423c599f1b6a2b4cde8,0x244454ab29bafb5bbb95f8f6de28fcc45e81b620,0x506226a43ddfffc0bfb48124e7b782232624430010b1...,52186.000000,False
49,2023-09-09T15:09:31.289Z,2023-09-09 15:02:25.000 UTC,0x557eae5968432ca52244f16bd364308777b49c63,0xc3fcab887a3f9b07776a47a2fbe452934cecac13,0xcd13c24468d36d45ad353156b8956a9b0b598e8d1372...,70000.000000,False
63,2023-09-09T15:50:12.825Z,2023-09-09 15:36:55.000 UTC,0x27d559ef0a9ab0d00dedbe5e0d5c215ec475ee24,0x4e4c24299bece3aff08b013d10023eb011394199,0xab810a4bac3fda9f2f5f6fa51818cdf0a7f9ad264216...,0.000037,False
65,2023-09-09T16:00:12.038Z,2023-09-09 13:32:15.000 UTC,0x9680ca3188117a40d6ce7c940020c7ac3afd58a4,0x865a0bef05fa813aae59ef12d88ace8eebef49d0,0x68a9e3e3c36c1b811beda8976d72e711d3fd156b9533...,0.000000,False
128,2023-09-10T07:07:53.959Z,2023-09-10 06:28:05.000 UTC,0xbd6e77e2b40e1613047479b0c7c720041d4e3f8f,0x138969cbb6b1346151986493bf82084eb98a5c2a,0x24203d9be19d0b0c109721eeb313afd9817cc16a2c45...,64257.000000,False


In [16]:
waitlist_01[waitlist_01["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True


### Delete non-elidgible nodes 

In [8]:
# Not enough stake
waitlist_02 = waitlist_01[waitlist_01['wxHOPR_balance'] >= 10000]
print(len(waitlist_02))
print(waitlist_02.dtypes)

122
registration_time      object
deployment_date        object
node_address           object
safe_address           object
deployment_tx_hash     object
wxHOPR_balance        float64
nr_nft                 object
dtype: object


In [18]:
waitlist_02[waitlist_02["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True


In [9]:
waitlist_02.loc[:, 'nr_nft'] = waitlist_02['nr_nft'].astype(bool)
print(waitlist_02['nr_nft'].value_counts())

nr_nft
True     102
False     20
Name: count, dtype: int64


In [10]:
# Users with a stake >= 30000 are eligible regardless of NR NFT
waitlist_03 = waitlist_02.copy()
waitlist_03.loc[:, 'eligible'] = np.where(waitlist_02.loc[:,'wxHOPR_balance'] >= 30000, True, False)

# Users with a stake < 30000 are eligible if they have a network registry NFT
waitlist_04 = waitlist_03.copy()
waitlist_04.loc[:, 'eligible'] = np.where((waitlist_03.loc[:,'wxHOPR_balance'] < 30000) & (waitlist_03.loc[:,'nr_nft'] == True), True, waitlist_03.loc[:, 'eligible'])

print(waitlist_04.loc[:, 'eligible'].value_counts())


eligible
True    122
Name: count, dtype: int64


In [11]:
waitlist_04[waitlist_04["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
283,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [12]:
waitlist_05 = waitlist_04[waitlist_04.loc[:,'eligible'] == True]
print(len(waitlist_05))

122


In [23]:
waitlist_05[waitlist_05["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [13]:
waitlist_05["node_address"] = waitlist_05["node_address"].str.strip()

In [14]:
# delete users that registered with their peer_id
valid_address = waitlist_05['node_address'].str.startswith('0x')
waitlist_06 = waitlist_05[valid_address]
print(len(waitlist_06))

119


In [15]:
waitlist_06[waitlist_06["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
283,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [33]:
waitlist_06[waitlist_06["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


### Sort Users 

In [16]:
nr_waitlist = waitlist_06[waitlist_06['nr_nft'] == True]
stake_waitlist =waitlist_06[waitlist_06['nr_nft'] == False]

print(len(nr_waitlist))
print(len(stake_waitlist))

99
20


In [17]:
nr_waitlist_01 = nr_waitlist.sort_values(by="deployment_date", ascending=True).reset_index(drop=True)
print(len(nr_waitlist_01))
display(nr_waitlist_01.head())

99


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-10-13T18:50:33.676Z,2023-09-07 11:24:55.000 UTC,0xb0117ae711702b8207bd9efc839c16c0901de471,0xca9e03270fb7483d405785ddc48cae57bedde30b,0xf3a17ad7cedb66a49c1b396f5d762e4717c69b94ac93...,13426.828286,True,True
1,2023-09-14T16:06:45.007Z,2023-09-14 16:03:05.000 UTC,0x10b31446016c83cb4f0a19e0de7d541c62857d10,0x88e1a25f59326430c654247a7d0abd104db40122,0xcbd22ca365bfacb814d57b08ab3bef75445a7ee26b44...,75000.000000,True,True
2,2023-09-14T16:20:56.588Z,2023-09-14 16:16:45.000 UTC,0xe4d33cbc725419c569017abf7f90098219407b2f,0xcc1b35a29f3dc918dc052614c8dbfeef1f3eb29d,0x1d310e5185d85f01260609b45c9cbffe768e72d0c212...,75000.000000,True,True
3,2023-09-14T16:44:05.837Z,2023-09-14 16:40:20.000 UTC,0xee33c72ebb9393693f60dc2ba01c48662cbef414,0xa0e6574bfac01d928bc84c6f41fff10d6e747115,0x087d5005e4adbb25dc2063afaeb1e61666c1252e3824...,75000.000000,True,True
4,2023-09-14T16:59:56.785Z,2023-09-14 16:52:30.000 UTC,0xf62ed730c314f77e76d383e185f34638ce034b90,0x4a696d506c96fb25f436359b08f93833fa338bb7,0x6582cf183723c4191b6390497b6807c84aa112bff711...,75000.000000,True,True


In [18]:
nr_waitlist_01[nr_waitlist_01["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
78,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [19]:
print(len(nr_waitlist_01))

99


In [20]:
stake_waitlist_01 = stake_waitlist.sort_values(by="wxHOPR_balance", ascending=False).reset_index(drop=True)
print(len(stake_waitlist))
display(stake_waitlist_01.head())

20


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-10-10T17:30:37.018Z,2023-10-10 10:36:15.000 UTC,0x7838771635cd80cee269085de751420a212cad55,0xd40cb88d9f3764056d9efb4074d98709e5e271a0,0xd71eec1a84e8c2d33b276ddcd35ff8ecd83927bc26ce...,75000.0,False,True
1,2023-09-15T16:45:05.138Z,2023-09-15 16:41:15.000 UTC,0x2b91e1aea65a805835ff19e919167fed3df25373,0x16d9965253dc787bcccd765f91a6c7c6cffb7216,0xb2cd269f1293594114fc085ff1b67d66a53b089a8052...,75000.0,False,True
2,2023-09-17T12:34:02.349Z,2023-09-17 12:29:40.000 UTC,0xa996eb759d44c8e2ea2de764ae031231678fe8b0,0x04f0e66eba222fa37dd9942ec1b4d6b793b1b676,0x1ba54e91d03583bd985b6b1e9e734b2a1e4c0b7b586b...,75000.0,False,True
3,2023-09-17T12:26:13.382Z,2023-09-17 12:23:05.000 UTC,0x02683d5a0526e2836e72bf689fd100eb9f8453b2,0xc827dcc5d10474830595e0679dd44e172b565dbc,0x3ea7d28e061b55b6fce08ffff53dc826bc48e8f14a5f...,75000.0,False,True
4,2023-10-10T10:30:42.740Z,2023-10-10 10:15:30.000 UTC,0xd0f7fc979f3802e439b908a40e20903369b55857,0x59b410c3e784133580cc1250fa6f4604aaaaf8c0,0x72aa0c805bd5c9f1a55e9491bdf3aa430ce23f00fc3d...,75000.0,False,True


In [21]:
print(len(stake_waitlist_01))

20


In [22]:
# Assuming you have two dataframes, nr_waitlist_01 and stake_waitlist_01
nr_chunk_size = 20
stake_chunk_size = 10

nr_index = 0  # Starting index for nr_waitlist_01
stake_index = 0  # Starting index for stake_waitlist_01

final_chunks = []  # List to store concatenated chunks

while nr_index < len(nr_waitlist_01) or stake_index < len(stake_waitlist_01):
    # Add a chunk of nr if there are more rows available
    if nr_index < len(nr_waitlist_01):
        nr_chunk = nr_waitlist_01.iloc[nr_index:nr_index + nr_chunk_size]
        final_chunks.append(nr_chunk)
        nr_index += nr_chunk_size

    # Add a chunk of stake if there are more rows available
    if stake_index < len(stake_waitlist_01):
        stake_chunk = stake_waitlist_01.iloc[stake_index:stake_index + stake_chunk_size]
        final_chunks.append(stake_chunk)
        stake_index += stake_chunk_size

# Concatenate the final chunks row-wise
final_waitlist = pd.concat(final_chunks, ignore_index=True)

display(final_waitlist.head())
print(len(final_waitlist))

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-10-13T18:50:33.676Z,2023-09-07 11:24:55.000 UTC,0xb0117ae711702b8207bd9efc839c16c0901de471,0xca9e03270fb7483d405785ddc48cae57bedde30b,0xf3a17ad7cedb66a49c1b396f5d762e4717c69b94ac93...,13426.828286,True,True
1,2023-09-14T16:06:45.007Z,2023-09-14 16:03:05.000 UTC,0x10b31446016c83cb4f0a19e0de7d541c62857d10,0x88e1a25f59326430c654247a7d0abd104db40122,0xcbd22ca365bfacb814d57b08ab3bef75445a7ee26b44...,75000.000000,True,True
2,2023-09-14T16:20:56.588Z,2023-09-14 16:16:45.000 UTC,0xe4d33cbc725419c569017abf7f90098219407b2f,0xcc1b35a29f3dc918dc052614c8dbfeef1f3eb29d,0x1d310e5185d85f01260609b45c9cbffe768e72d0c212...,75000.000000,True,True
3,2023-09-14T16:44:05.837Z,2023-09-14 16:40:20.000 UTC,0xee33c72ebb9393693f60dc2ba01c48662cbef414,0xa0e6574bfac01d928bc84c6f41fff10d6e747115,0x087d5005e4adbb25dc2063afaeb1e61666c1252e3824...,75000.000000,True,True
4,2023-09-14T16:59:56.785Z,2023-09-14 16:52:30.000 UTC,0xf62ed730c314f77e76d383e185f34638ce034b90,0x4a696d506c96fb25f436359b08f93833fa338bb7,0x6582cf183723c4191b6390497b6807c84aa112bff711...,75000.000000,True,True


119


In [23]:
print(len(final_waitlist))

119


In [44]:
final_waitlist[final_waitlist["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
125,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [25]:
### sanity checks

# check users who already got access are not in the waitlist
print(pd.Series(final_waitlist['safe_address']).isin(list(network_waitlist_data[network_waitlist_data['Eligibility'] == "yes"]["Safe address"])).value_counts())

# check that all users who were already in the waitlist are indeed in the waitlist
print(pd.Series(final_waitlist['safe_address']).isin(list(network_waitlist_data[network_waitlist_data['Eligibility'].isnull()]["Safe address"])).value_counts())

# check that no duplicates are in the waitlist
print((final_waitlist.duplicated(subset=['safe_address'], keep='last')).value_counts())

safe_address
False    119
Name: count, dtype: int64
safe_address
True     114
False      5
Name: count, dtype: int64
False    119
Name: count, dtype: int64


In [32]:
final_waitlist[final_waitlist["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible


### Save final waitlist 

In [26]:
final_waitlist.to_excel('final_waitlist.xlsx', index=False)